In [1]:
'''
画出红色图形的轮廓，其中很多参数待调，计划通过TK设立滑块来实时调节参数
'''

import cv2
import numpy as np

In [3]:
cap = cv2.VideoCapture(0)   # 读取摄像头

while True:
    # 读取下一帧解码本帧
    ret,im = cap.read()

    # 用numpy建立一个和视频一样长宽的全黑画布
    # 便于后边将图像画于画布上
    pri = np.zeros(im.shape[:2])

    # 将图像转化到HSV色彩空间格式，便于提取颜色
    hsv = cv2.cvtColor(im,cv2.COLOR_BGR2HSV)

    # 制作透过红色的蒙版
    lower_red = np.array([160,110,170])
    upper_red = np.array([180,255,255])
    maskR = cv2.inRange(hsv,lower_red,upper_red)

    # 将蒙版与图像结合，得到只透红色的图像
    res = cv2.bitwise_and(im,im,mask=maskR)

    # 画出图像中红色图形的外轮廓
    gray = cv2.cvtColor(res,cv2.COLOR_BGR2GRAY)    # 将处理后的图像转换为灰度图，便于二值化
    re,thresh = cv2.threshold(gray,1,1,cv2.THRESH_TOZERO)   # 将图像二值化，此函数只接受灰度图
    image,contours,hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    # 在二值化的基础上找出轮廓
    # image为thresh图像，contours为轮廓点的坐标，hierarchy为轮廓索引
    #目前为止，hierarchy属性显得非常鸡肋，因为是动态视频，每次屏幕刷新索引号都不一样，所以无法应用索引号去噪
    
    if contours == []:
        continue
    else:
        M = cv2.moments(contours[0])
        if int(M['m00']) == 0:
            continue
        else:
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])

    # 在画布上画出轮廓
    pri = np.zeros(im.shape[:2])
    img = cv2.drawContours(pri,contours,-1,(255,255,255),1)
    cv2.circle(img,(cx,cy),2,(255,255,255),-1)
    cv2.imshow("video",img)

    if cv2.waitKey(1) &0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()